In [ ]:
"""
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
"""

# Cycle GAN Part 2

I trained a cycle GAN in another notbook, and saver a lot of checkpoints, this is because images from some epochs are for my taste better than others and not all the times improving (maybe i should have used some kind of modifier for hyperparametes as epochs went on but i did`t and i belive images are good enough). 

So anyway in went out of space for creating the subbmision file. This is a small notbook loading network again (did not save entire model becasue of HDD space) and using weights from my favorite epoch and creating file for submission.

In [ ]:
import os
import keras
import glob
import cv2

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

from PIL import Image
from sklearn.cluster import KMeans
from keras import layers, ops

# CycleGAN 1. Data transformation


In [ ]:
#Read data
# Get lists of file paths for each domain
monet_paths = glob.glob("/kaggle/input/gan-getting-started/monet_jpg/*.jpg")
photo_paths = glob.glob("/kaggle/input/gan-getting-started/photo_jpg/*.jpg")

# Create datasets from file paths
monet_ds = tf.data.Dataset.from_tensor_slices(monet_paths)
photo_ds = tf.data.Dataset.from_tensor_slices(photo_paths)

# Function to load an image, convert pixel values to [-1, 1], and assign a label.
def load_monet_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)  # Scales to [0, 1]
    image = (image - 0.5) * 2  # Now in [-1, 1]
    return image

def load_photo_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)  # Scales to [0, 1]
    image = (image - 0.5) * 2  # Now in [-1, 1]
    return image

# Map the functions to load and decode images with labels
monet_ds = (
    monet_ds.map(load_monet_image, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(buffer_size=1000)
)

photo_ds = (
     photo_ds.map(load_photo_image, num_parallel_calls=tf.data.AUTOTUNE)
     .shuffle(buffer_size=1000)
 )

# Organize datasets in a dictionary similar to CycleGAN's "trainA" and "trainB"
dataset = {
    "trainA": monet_ds,
    "trainB": photo_ds
}


In [ ]:
#buffer and batches
buffer_size = 256
batch_size = 1

#loading datasets
train_monet, train_photos = dataset["trainA"], dataset["trainB"]
train_monet = (
    train_monet.map(lambda img: img, num_parallel_calls=tf.data.AUTOTUNE)
    .cache()
    .shuffle(buffer_size)
    .batch(batch_size)
)
train_photos = (
    train_photos.map(lambda img: img, num_parallel_calls=tf.data.AUTOTUNE)
    .cache()
    .shuffle(buffer_size)
    .batch(batch_size)
)

# Cycle GAN 2. Constructing NN

In [ ]:
#Important Variables
autotune = tf.data.AUTOTUNE
#Kernel_init
kernel_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
# Gamma initializer for instance normalization.
gamma_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
#img size
input_img_size = (256, 256, 3)


## Basic Functions

In [ ]:
#Building blocks used in the CycleGAN generators and discriminators

"""
-------------------
Block of RNN
Recieves x, type of activation function, kernel init, size, stride, uses padding same, gamma init an bias as false
    
    1. Convolution + normalization + activation funtion
    2. Convolution + normalization
    3. Residual or skip conection from initial x to final x

dim = x.shape[-1]: Number of channels of RGB input - In a residual block, the goal is to learn a transformation that can be added back to the original input (the skip connection). For this addition to work, the input and the transformed output must have the same shape. By using x.shape[-1] (the number of channels in the input) as the number of filters for the convolutional layers, you ensure that the output of the block has the same number of channels as the input, making the element-wise addition possible. This design is a standard practice in architectures like ResNet and CycleGAN, where preserving the feature dimensions across a block helps in learning the residual mapping effectively.

"""

def residual_block(
    x,
    activation,
    kernel_initializer=kernel_init,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding="same",  # Use 'same' padding instead of reflection padding
    gamma_initializer=gamma_init,
    use_bias=False,
    ):
    
    dim = x.shape[-1] 
    input_tensor = x

    # First convolutional layer with same padding
    x = layers.Conv2D(
        dim, # dim is the number of filters
        kernel_size,
        strides=strides,
        kernel_initializer=kernel_initializer,
        padding=padding,
        use_bias=use_bias,
    )(input_tensor)
    x = keras.layers.GroupNormalization(groups=1, gamma_initializer=gamma_initializer)(x)
    x = activation(x)

    # Second convolutional layer with same padding
    x = layers.Conv2D(
        dim,
        kernel_size,
        strides=strides,
        kernel_initializer=kernel_initializer,
        padding=padding,
        use_bias=use_bias,
    )(x)
    x = keras.layers.GroupNormalization(groups=1, gamma_initializer=gamma_initializer)(x)

    # Add the input tensor (residual connection)
    x = layers.add([input_tensor, x])
    return x
    
"""
-------------------
Downsample function 
Recieves x, type of activation function, kernel init, size, stride, uses padding same, gamma init an bias as false
    
    1. Convolution + normalization (with a bigger stride to downsample)
"""

def downsample(
    x,
    filters,
    activation,
    kernel_initializer=kernel_init,
    kernel_size=(3, 3),
    strides=(2, 2),
    padding="same",
    gamma_initializer=gamma_init,
    use_bias=False,
):
    x = layers.Conv2D(
        filters,
        kernel_size,
        strides=strides,
        kernel_initializer=kernel_initializer,
        padding=padding,
        use_bias=use_bias,
    )(x)
    x = keras.layers.GroupNormalization(groups=1, gamma_initializer=gamma_initializer)(
        x
    )
    if activation:
        x = activation(x)
    return x
    
"""
-------------------
Upsample function 
Recieves x, type of activation function, kernel init, size, stride, uses padding same, gamma init an bias as false
    
    1. Inverse - Convolution + normalization (with a bigger stride to downsample)
"""

def upsample(
    x,
    filters,
    activation,
    kernel_size=(3, 3),
    strides=(2, 2),
    padding="same",
    kernel_initializer=kernel_init,
    gamma_initializer=gamma_init,
    use_bias=False,
):
    x = layers.Conv2DTranspose(
        filters,
        kernel_size,
        strides=strides,
        padding=padding,
        kernel_initializer=kernel_initializer,
        use_bias=use_bias,
    )(x)
    x = keras.layers.GroupNormalization(groups=1, gamma_initializer=gamma_initializer)(
        x
    )
    if activation:
        x = activation(x)
    return x

## RESNet Generator (Function)

In [ ]:
"""
--------------
1. Convolution + Normalization + activation
2. Downsampling - uses funtion defined previously
3. Residual blocks - uses funtion defined previously
3. Upsampling - uses funtion defined previously
4. Final convolution and activtion

"""
def get_resnet_generator(
    filters=64,
    num_downsampling_blocks=2,
    num_residual_blocks=9,
    num_upsample_blocks=2,
    gamma_initializer=gamma_init,
    name=None,
):
    img_input = layers.Input(shape=input_img_size, name=name + "_img_input")
    x = img_input
    x = layers.Conv2D(filters, (7, 7), padding="same", kernel_initializer=kernel_init, use_bias=False)(x)
    x = keras.layers.GroupNormalization(groups=1, gamma_initializer=gamma_initializer)(x)
    x = layers.Activation("relu")(x)

    # Downsampling
    for _ in range(num_downsampling_blocks):
        filters *= 2
        x = downsample(x, filters=filters, activation=layers.Activation("relu"))

    # Residual blocks
    for _ in range(num_residual_blocks):
        x = residual_block(x, activation=layers.Activation("relu"))

    # Upsampling
    for _ in range(num_upsample_blocks):
        filters //= 2
        x = upsample(x, filters, activation=layers.Activation("relu"))

    # Final block
    x = layers.Conv2D(3, (7, 7), padding="same")(x)
    x = layers.Activation("tanh")(x)

    model = keras.models.Model(img_input, x, name=name)
    return model

## Discriminator (Function)

In [ ]:
def get_discriminator(filters=64, kernel_initializer=kernel_init, num_downsampling=3, name=None):
    img_input = layers.Input(shape=input_img_size, name=name + "_img_input")
    x = layers.Conv2D(filters,(4, 4),strides=(2, 2), padding="same", kernel_initializer=kernel_initializer,)(img_input)
    x = layers.LeakyReLU(0.2)(x)

    num_filters = filters
    for num_downsample_block in range(3):
        num_filters *= 2
        if num_downsample_block < 2:
            x = downsample(
                x,
                filters=num_filters,
                activation=layers.LeakyReLU(0.2),
                kernel_size=(4, 4),
                strides=(2, 2),
            )
        else:
            x = downsample(
                x,
                filters=num_filters,
                activation=layers.LeakyReLU(0.2),
                kernel_size=(4, 4),
                strides=(1, 1),
            )

    x = layers.Conv2D(1, (4, 4), strides=(1, 1), padding="same", kernel_initializer=kernel_initializer)(x)

    model = keras.models.Model(inputs=img_input, outputs=x, name=name)
    return model

## Creating Networks 

In [ ]:
#
#Get the generators
gen_G = get_resnet_generator(name="generator_G") # Translate features from monet to photos
gen_F = get_resnet_generator(name="generator_F") # reverse proces photos to monet

# Get the discriminators
disc_X = get_discriminator(name="discriminator_X") #Discrimination on monet real or fake
disc_Y = get_discriminator(name="discriminator_Y") #Discriminator on photo real or fake

# Cycle GAN 3. Teaching to paint class CycleGan

In [ ]:
class CycleGan(keras.Model):
    def __init__(
        self,
        generator_G,
        generator_F,
        discriminator_X,
        discriminator_Y,
        lambda_cycle=10.0,
        lambda_identity=0.5,
    ):
        super().__init__() # calls init from parent class (keras.Model)
        self.gen_G = generator_G
        self.gen_F = generator_F
        self.disc_X = discriminator_X
        self.disc_Y = discriminator_Y
        self.lambda_cycle = lambda_cycle
        self.lambda_identity = lambda_identity

    def call(self, inputs):
        return (
            self.disc_X(inputs),
            self.disc_Y(inputs),
            self.gen_G(inputs),
            self.gen_F(inputs),
        )

    def compile(
        self,
        gen_G_optimizer,
        gen_F_optimizer,
        disc_X_optimizer,
        disc_Y_optimizer,
        gen_loss_fn,
        disc_loss_fn,
    ):
        super().compile()
        self.gen_G_optimizer = gen_G_optimizer
        self.gen_F_optimizer = gen_F_optimizer
        self.disc_X_optimizer = disc_X_optimizer
        self.disc_Y_optimizer = disc_Y_optimizer
        self.generator_loss_fn = gen_loss_fn
        self.discriminator_loss_fn = disc_loss_fn
        self.cycle_loss_fn = keras.losses.MeanAbsoluteError()
        self.identity_loss_fn = keras.losses.MeanAbsoluteError()

    def train_step(self, batch_data):
        # x is monet and y is photo
        real_x, real_y = batch_data

        # For CycleGAN, we need to calculate different
        # kinds of losses for the generators and discriminators.
        # We will perform the following steps here:
        #
        # 1. Pass real images through the generators and get the generated images
        # 2. Pass the generated images back to the generators to check if we
        #    can predict the original image from the generated image.
        # 3. Do an identity mapping of the real images using the generators.
        # 4. Pass the generated images in 1) to the corresponding discriminators.
        # 5. Calculate the generators total loss (adversarial + cycle + identity)
        # 6. Calculate the discriminators loss
        # 7. Update the weights of the generators
        # 8. Update the weights of the discriminators
        # 9. Return the losses in a dictionary

        with tf.GradientTape(persistent=True) as tape:
            # monet to fake photo
            fake_y = self.gen_G(real_x, training=True)
            # photo to fake monet -> y2x
            fake_x = self.gen_F(real_y, training=True)

            # Cycle (monet to fake photo to fake monet): x -> y -> x
            cycled_x = self.gen_F(fake_y, training=True)
            # Cycle (photo to fake moent to fake photo) y -> x -> y
            cycled_y = self.gen_G(fake_x, training=True)

            # Identity mapping
            same_x = self.gen_F(real_x, training=True)
            same_y = self.gen_G(real_y, training=True)

            # Discriminator output
            disc_real_x = self.disc_X(real_x, training=True)
            disc_fake_x = self.disc_X(fake_x, training=True)

            disc_real_y = self.disc_Y(real_y, training=True)
            disc_fake_y = self.disc_Y(fake_y, training=True)

            # Generator adversarial loss
            gen_G_loss = self.generator_loss_fn(disc_fake_y)
            gen_F_loss = self.generator_loss_fn(disc_fake_x)

            # Generator cycle loss
            cycle_loss_G = self.cycle_loss_fn(real_y, cycled_y) * self.lambda_cycle
            cycle_loss_F = self.cycle_loss_fn(real_x, cycled_x) * self.lambda_cycle

            # Generator identity loss
            id_loss_G = (
                self.identity_loss_fn(real_y, same_y)
                * self.lambda_cycle
                * self.lambda_identity
            )
            id_loss_F = (
                self.identity_loss_fn(real_x, same_x)
                * self.lambda_cycle
                * self.lambda_identity
            )

            # Total generator loss
            total_loss_G = gen_G_loss + cycle_loss_G + id_loss_G
            total_loss_F = gen_F_loss + cycle_loss_F + id_loss_F

            # Discriminator loss
            disc_X_loss = self.discriminator_loss_fn(disc_real_x, disc_fake_x)
            disc_Y_loss = self.discriminator_loss_fn(disc_real_y, disc_fake_y)

        # Get the gradients for the generators
        grads_G = tape.gradient(total_loss_G, self.gen_G.trainable_variables)
        grads_F = tape.gradient(total_loss_F, self.gen_F.trainable_variables)

        # Get the gradients for the discriminators
        disc_X_grads = tape.gradient(disc_X_loss, self.disc_X.trainable_variables)
        disc_Y_grads = tape.gradient(disc_Y_loss, self.disc_Y.trainable_variables)

        # Update the weights of the generators
        self.gen_G_optimizer.apply_gradients(
            zip(grads_G, self.gen_G.trainable_variables)
        )
        self.gen_F_optimizer.apply_gradients(
            zip(grads_F, self.gen_F.trainable_variables)
        )

        # Update the weights of the discriminators
        self.disc_X_optimizer.apply_gradients(
            zip(disc_X_grads, self.disc_X.trainable_variables)
        )
        self.disc_Y_optimizer.apply_gradients(
            zip(disc_Y_grads, self.disc_Y.trainable_variables)
        )

        return {
            "G_loss": total_loss_G,
            "F_loss": total_loss_F,
            "D_X_loss": disc_X_loss,
            "D_Y_loss": disc_Y_loss,
        }
        

## Loss Functions

In [ ]:
# Loss function for evaluating adversarial loss
adv_loss_fn = keras.losses.MeanSquaredError()

# Define the loss function for the generators
def generator_loss_fn(fake):
    fake_loss = adv_loss_fn(ops.ones_like(fake), fake)
    return fake_loss


# Define the loss function for the discriminators
def discriminator_loss_fn(real, fake):
    real_loss = adv_loss_fn(ops.ones_like(real), real)
    fake_loss = adv_loss_fn(ops.zeros_like(fake), fake)
    return (real_loss + fake_loss) * 0.5

## Compile and load weights (previously trained)

In [ ]:
# Create cycle gan model
"""
---------------
Model
---------------
"""
cycle_gan_model = CycleGan(
    generator_G=gen_G, generator_F=gen_F, discriminator_X=disc_X, discriminator_Y=disc_Y
)

# Compile (final configuration) of the model

cycle_gan_model.compile(
    gen_G_optimizer=keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5),
    gen_F_optimizer=keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5),
    disc_X_optimizer=keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5),
    disc_Y_optimizer=keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5),
    gen_loss_fn=generator_loss_fn,
    disc_loss_fn=discriminator_loss_fn,
)

"""
----------------
Model from checkpoint 
---------------
"""
cycle_gan_model.load_weights("/kaggle/input/weights/cyclegan71_checkpoint.weights.h5")


# Results - Submission

In [ ]:
# Show painted the landscapes
_, ax = plt.subplots(4, 2, figsize=(10, 15))
for i, img in enumerate(train_photos.take(4)):
    prediction = cycle_gan_model.gen_F(img, training=False)[0].numpy()
    prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
    img = (img[0] * 127.5 + 127.5).numpy().astype(np.uint8)

    ax[i, 0].imshow(img)
    ax[i, 1].imshow(prediction)
    ax[i, 0].set_title("Input image")
    ax[i, 1].set_title("Translated image")
    ax[i, 0].axis("off")
    ax[i, 1].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
import os
import numpy as np
import PIL
import shutil

# Create the directory if it doesn't exist
output_dir = "../images"
os.makedirs(output_dir, exist_ok=True)

i = 1
for img in train_photos:
    prediction = cycle_gan_model.gen_F(img, training=False)[0].numpy()
    prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
    im = PIL.Image.fromarray(prediction)
    
    save_path = os.path.join(output_dir, f"{i}.jpg")
    im.save(save_path)

    if i % 200 == 0:
        print(f"{i}/7038")
    
    i += 1

# Create a ZIP archive of the images directory
shutil.make_archive("/kaggle/working/images", 'zip', output_dir)


In [ ]:
 # rm -rf /kaggle/working/